# Step 2: Extract Marking Scheme to Excel

Use Gemini to parse the Word marking scheme into structured Excel sheets with error handling, validation, and progress tracking.

**Features:**
- ✅ Comprehensive error handling and validation
- ✅ Robust progress tracking and logging
- ✅ Improved markdown formatting and structure
- ✅ Robust file handling and backup
- ✅ Detailed validation and quality checks
- ✅ Professional output formatting

Configure the exam `prefix` and dataset folder in the next cell before running.


In [1]:
from grading_utils import setup_paths, init_gemini_client
from agents.marking_scheme_agent.agent import extract_marking_scheme_with_ai, verify_marking_scheme_with_ai
from google.genai import types
import mammoth
import html2text
from IPython.display import Markdown, display, clear_output
from pydantic import BaseModel, Field
from typing import List
import json
import pandas as pd
from termcolor import colored
import os
import shutil
import logging
from datetime import datetime
import docx
from docx import Document
from docx.shared import RGBColor
from docx.enum.dml import MSO_THEME_COLOR_INDEX
import re

def add_hyperlink(paragraph, text, url):
    """Adds a clickable hyperlink to a paragraph"""
    # This gets access to the document.xml.rels file and gets a new relation id
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id,)

    # Create a w:r element
    new_run = docx.oxml.shared.OxmlElement('w:r')

    # Create a w:rPr element
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Add color element, default to blue
    c = docx.oxml.shared.OxmlElement('w:color')
    c.set(docx.oxml.shared.qn('w:val'), '0000FF')
    rPr.append(c)

    # Add underline element
    u = docx.oxml.shared.OxmlElement('w:u')
    u.set(docx.oxml.shared.qn('w:val'), 'single')
    rPr.append(u)

    # Join all the xml elements together and add the text
    new_run.append(rPr)
    t = docx.oxml.shared.OxmlElement('w:t')
    t.text = text
    new_run.append(t)
    hyperlink.append(new_run)

    paragraph._p.append(hyperlink)

    return hyperlink

def add_formatted_text(paragraph, text):
    """Parses markdown-style links [title](url) and adds them to paragraph"""
    # Simple regex to find [title](url)
    pattern = r'\[(.*?)\]\((.*?)\)'
    last_end = 0
    
    for match in re.finditer(pattern, text):
        # Add plain text before the link
        paragraph.add_run(text[last_end:match.start()])
        
        # Add the hyperlink
        link_text = match.group(1)
        link_url = match.group(2)
        add_hyperlink(paragraph, link_text, link_url)
        
        last_end = match.end()
    
    # Add remaining plain text
    paragraph.add_run(text[last_end:])

# Robust logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Robust Step 2: Extract Marking Scheme to Excel initialized")
print(f"✓ Session started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Configuration
prefix = "VTC Test"  # Change this to your exam name
dataset = "sample"   # Change to your dataset folder

# Setup paths with validation
try:
    paths = setup_paths(prefix, dataset)
    marking_scheme_word_file = f"../{dataset}/{prefix} Marking Scheme.docx"
    marking_scheme_excel_file = paths["marking_scheme_file"]
    
    # Validate input file exists
    if not os.path.exists(marking_scheme_word_file):
        raise FileNotFoundError(f"Marking scheme file not found: {marking_scheme_word_file}")
    
    logger.info(f"✓ Input file validated: {marking_scheme_word_file}")
    logger.info(f"✓ Output file will be: {marking_scheme_excel_file}")
    
    print(f"📁 Input: {marking_scheme_word_file}")
    print(f"📁 Output: {marking_scheme_excel_file}")
    
except Exception as e:
    logger.error(f"❌ Setup failed: {e}")
    raise

Bad pipe message: %s [b' 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Sa']
Bad pipe message: %s [b'ri/537.36\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/', b'ng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nAccept-Encoding: gzip, deflate, br, zstd\r\nA']
Bad pipe message: %s [b'ept-Language: en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7\r\nPriority: u=0, i\r\nReferer: https://studio.fireb', b'e.google.com/\r\nSec-Ch-Ua: "Google Chrome";v="143", "Chromium";v="143", "Not A(Brand";v="24"\r\nSec', b'h-Ua-Arch: "x86"\r\nSec-Ch-Ua-Bitness: "64"\r\nS', b'-Ch-Ua-Form-Factors: "Desktop"\r\nSec-Ch-Ua-Full-Version: "143.0.7499.170"\r\nSec-Ch-Ua-Full-Version-Lis', b' "Google Chrome";v="143.0.7499.170", "Chromium";v="143.0.7499.170", "Not A(Brand";v="24.0.0.0"\r\nSec-Ch-Ua-Mobile: ?']
Bad pipe message: %s [b'\nSec-Ch-Ua-Model: ""\r\nSec-Ch-Ua-Platform: "Wind', b's"\r\nSec-Ch-Ua-Platform-Version: "19.0.0"\r\nSec-Ch-Ua-Wow64: ?

✅ Robust Step 2: Extract Marking Scheme to Excel initialized
✓ Session started at: 2026-01-07 14:19:01
📁 Input: ../sample/VTC Test Marking Scheme.docx
📁 Output: ../sample/VTC Test Marking Scheme.xlsx


In [2]:
# Robust Gemini client initialization with error handling
try:
    client = init_gemini_client()
    logger.info("✓ Gemini client initialized successfully")
    print("🤖 Gemini AI client ready")
except Exception as e:
    logger.error(f"❌ Failed to initialize Gemini client: {e}")
    print(f"❌ Gemini initialization failed: {e}")
    raise

2026-01-07 14:19:02,177 - INFO - ✓ Gemini client initialized successfully


✓ Vertex AI Express Mode initialized
🤖 Gemini AI client ready


In [3]:
# Robust document processing with comprehensive error handling
def process_word_document(file_path):
    """Process Word document with error handling and validation"""
    try:
        logger.info(f"Processing Word document: {file_path}")
        
        # Validate file exists and is readable
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        file_size = os.path.getsize(file_path)
        logger.info(f"File size: {file_size:,} bytes")
        
        # Convert .docx to HTML using mammoth
        with open(file_path, "rb") as docx_file:
            result = mammoth.convert_to_html(docx_file)
            html_content = result.value
            
            # Check for conversion warnings
            if result.messages:
                logger.warning(f"Mammoth conversion warnings: {len(result.messages)}")
                for msg in result.messages[:5]:  # Show first 5 warnings
                    logger.warning(f"  - {msg}")
        
        # Convert HTML to markdown using html2text
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.body_width = 0  # Don't wrap text
        h.ignore_images = True
        h.ignore_tables = False
        
        markdown_content = h.handle(html_content)
        
        # Validate content
        if not markdown_content.strip():
            raise ValueError("No content extracted from document")
        
        content_length = len(markdown_content)
        logger.info(f"✓ Extracted {content_length:,} characters of markdown content")
        
        return markdown_content, html_content
        
    except Exception as e:
        logger.error(f"❌ Document processing failed: {e}")
        raise

# Process the document
try:
    markdown_content, html_content = process_word_document(marking_scheme_word_file)
    print("✅ Document processed successfully")
    print(f"📄 Content length: {len(markdown_content):,} characters")
    
    # Display preview of markdown content
    print("\n📋 Document Preview:")
    display(Markdown(markdown_content[:1000] + "..." if len(markdown_content) > 1000 else markdown_content))
    
except Exception as e:
    print(f"❌ Document processing failed: {e}")
    raise


2026-01-07 14:19:02,237 - INFO - Processing Word document: ../sample/VTC Test Marking Scheme.docx
2026-01-07 14:19:02,245 - INFO - File size: 18,126 bytes


2026-01-07 14:19:02,520 - INFO - ✓ Extracted 4,271 characters of markdown content


✅ Document processed successfully
📄 Content length: 4,271 characters

📋 Document Preview:


**Detailed Marking Scheme (0-10 Scale)**

Use the following rubric to determine the score for each q.

**Q1: The Role of VTC**  
The VTC is the largest provider of **VPET** in Hong Kong. Briefly explain what **VPET** stands for and why it is important for Hong Kong’s workforce development.

  * **Answer:** VPET stands for **Vocational and Professional Education and Training**. It is important because it provides students with practical skills and specialized knowledge needed by industries, ensuring Hong Kong has a skilled labor force to support the economy.
  * **Marking Breakdown:**
    * **[2 marks]** Correctly stating "Vocational and Professional Education and Training".
    * **[4 marks]** Explaining that it focuses on _practical skills_ or _specialized trades_.
    * **[4 marks]** Explaining the benefit to the workforce (reducing skills gap, employment readiness).



**Q2: Member Institutions**  
Compare **IVE (Hong Kong Institute of Vocational Education)** and **THEi (Technologic...

In [4]:
# Execute AI extraction using the imported agent
try:
    print("🤖 Processing document with Gemini AI...")
    # extract_marking_scheme_with_ai is imported from agents.marking_scheme_agent.agent
    questions_data, general_guide = await extract_marking_scheme_with_ai(markdown_content)
    
    print(f"✅ AI extraction successful!")
    print(f"📊 Extracted {len(questions_data)} questions")
    if general_guide:
        print(f"📋 General grading guide: {len(general_guide)} characters")
    
except Exception as e:
    print(f"❌ AI extraction failed: {e}")
    raise


2026-01-07 14:19:02,581 - INFO - Marking scheme cache hit


🤖 Processing document with Gemini AI...
✅ AI extraction successful!
📊 Extracted 5 questions
📋 General grading guide: 491 characters


In [5]:
# Robust validation and data processing
def validate_and_process_questions(questions_data, general_guide):
    """Comprehensive validation and processing of extracted questions"""
    try:
        logger.info("Validating extracted questions...")
        
        # Comprehensive validation
        validation_errors = []
        warnings = []
        
        if not questions_data:
            validation_errors.append("No questions extracted from document")
            
        for i, question in enumerate(questions_data):
            q_num = question.get('question_number', f'Question {i+1}')
            
            # Validate required fields
            if not question.get('question_text', '').strip():
                validation_errors.append(f"{q_num}: Missing question text")
            
            if not question.get('marking_scheme', '').strip():
                validation_errors.append(f"{q_num}: Missing marking scheme")
            
            # Validate marks
            marks = question.get('marks', 0)
            if not isinstance(marks, int) or marks <= 0:
                validation_errors.append(f"{q_num}: Invalid marks value ({marks})")
            elif marks > 50:
                warnings.append(f"{q_num}: High marks value ({marks}) - please verify")
        
        # Report validation results
        if validation_errors:
            print("\n" + "="*60)
            print(colored("❌ VALIDATION ERRORS DETECTED!", "red", attrs=['bold']))
            print("="*60)
            for error in validation_errors:
                print(colored(f"  • {error}", "red"))
            print("="*60)
            raise ValueError(f"Validation failed: {len(validation_errors)} error(s) found")
        
        if warnings:
            print("\n" + "⚠️  VALIDATION WARNINGS:")
            for warning in warnings:
                print(colored(f"  • {warning}", "yellow"))
            print()
        
        # Process questions - append general guide if available
        if general_guide and general_guide.strip():
            logger.info("Appending general grading guide to marking schemes")
            for question in questions_data:
                question['marking_scheme'] = f"{question['marking_scheme']}\n\n---\n\n**General Grading Guide:**\n{general_guide}"
        
        # Create DataFrame with formatting
        df = pd.DataFrame(questions_data)
        
        # Calculate statistics
        total_questions = len(questions_data)
        total_marks = df['marks'].sum()
        avg_marks = df['marks'].mean()
        
        logger.info(f"✓ Validation passed: {total_questions} questions, {total_marks} total marks")
        
        print(colored("✅ VALIDATION SUCCESSFUL!", "green", attrs=['bold']))
        print(f"📊 Questions: {total_questions}")
        print(f"📊 Total marks: {total_marks}")
        print(f"📊 Average marks per question: {avg_marks:.1f}")
        
        return df, total_questions, total_marks
        
    except Exception as e:
        logger.error(f"❌ Validation failed: {e}")
        raise

# Execute validation and processing
try:
    df, total_questions, total_marks = validate_and_process_questions(questions_data, general_guide)
    
    # Display results
    print("\n📋 Extracted Questions:")
    display(df[['question_number', 'question_text', 'marks']].head(10))
    
    if len(df) > 10:
        print(f"... and {len(df) - 10} more questions")
    
    # --- NEW VERIFICATION STEP ---
    print("\n🤖 Verifying content with Google Search Grounding...")
    verification_items, verification_general_feedback = await verify_marking_scheme_with_ai(questions_data)
    
    # Display Verification Results
    print(f"\n✅ Verification Complete!")
    if verification_general_feedback:
        print("📋 General Feedback:")
        display(Markdown(verification_general_feedback))
        print("\n")
        
    if verification_items:
        v_df = pd.DataFrame(verification_items)
        display(v_df[['question_number', 'is_correct', 'feedback', 'suggestion']])
    else:
        print("⚠️ No verification items returned.")
        v_df = pd.DataFrame() # Empty DataFrame
    
except Exception as e:
    print(f"❌ Validation failed: {e}")
    raise


2026-01-07 14:19:02,690 - INFO - Validating extracted questions...
2026-01-07 14:19:02,697 - INFO - Appending general grading guide to marking schemes
2026-01-07 14:19:02,706 - INFO - ✓ Validation passed: 5 questions, 50 total marks


✅ VALIDATION SUCCESSFUL!
📊 Questions: 5
📊 Total marks: 50
📊 Average marks per question: 10.0

📋 Extracted Questions:


,question_number,question_text,marks
0,Q1,The VTC is the largest provider of **VPET** in...,10
1,Q2,Compare **IVE (Hong Kong Institute of Vocation...,10
2,Q3,"VTC emphasizes the **"" Think and Do""** approac...",10
3,Q4,If a Secondary 6 student does **not** achieve ...,10
4,Q5,Why does the VTC collaborate closely with indu...,10


2026-01-07 14:19:02,755 - INFO - Starting sequential marking scheme verification...
2026-01-07 14:19:02,759 - INFO - AI execution attempt 1/3 for marking_scheme_verifier



🤖 Verifying content with Google Search Grounding...


2026-01-07 14:19:03,283 - INFO - Sending out request, model: gemini-3-flash-preview, backend: GoogleLLMVariant.VERTEX_AI, stream: False
2026-01-07 14:19:03,290 - INFO - AFC is enabled with max remote calls: 10.
2026-01-07 14:19:24,581 - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/publishers/google/models/gemini-3-flash-preview:generateContent "HTTP/1.1 200 OK"
2026-01-07 14:19:24,588 - INFO - Response received from the model.
2026-01-07 14:19:24,772 - INFO - Sending out request, model: gemini-3-flash-preview, backend: GoogleLLMVariant.VERTEX_AI, stream: False
2026-01-07 14:19:24,775 - INFO - AFC is enabled with max remote calls: 10.
2026-01-07 14:20:19,374 - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/publishers/google/models/gemini-3-flash-preview:generateContent "HTTP/1.1 200 OK"
2026-01-07 14:20:19,384 - INFO - Response received from the model.
2026-01-07 14:20:19,389 - INFO - ✓ Sequential verification completed: 5 items checked



✅ Verification Complete!
📋 General Feedback:


The marking scheme is highly accurate and ready for use. The terminology matches current Hong Kong educational policy (VPET) and VTC’s institutional structure. No factual errors were found. This evaluation confirms that the marking scheme is factually accurate and aligns with the official standards and terminology of the Vocational Training Council (VTC) in Hong Kong. [vtc.edu.hk](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQGfYjWT3BilK0Z-osBdBJkrXPQjTZDsS96PhghiYYnUHdoFpqFUpVSQHZDE4dIKCXYhvEE8x1Z4-BwC-kdHBBBLMTaY8h9bLVoJhUOxj-2i2_Jt848kCldR_bb5ZQ3CyZuT4EUrO9Yj)

,question_number,is_correct,feedback,suggestion
0,Q1,True,Terminology is 100% correct. VTC is officially...,The marking scheme is excellent. To make it ev...
1,Q2,True,Correctly distinguishes between the sub-degree...,The answer is correct. You might add that THEi...
2,Q3,True,This is the official VTC motto. The explanatio...,The marking breakdown is well-balanced. Ensure...
3,Q4,True,"DFS and DVE are the standard ""bridge"" programm...",The answer is accurate. Note that successful c...
4,Q5,True,"Benefits listed (internships, equipment) are c...","The examples provided (internships, equipment)..."


In [6]:
# Robust Excel export with comprehensive formatting and backup
def create_excel_report(df, marking_scheme_excel_file, total_questions, total_marks, verification_df=None, verification_feedback=""):
    """Create comprehensive Excel report with multiple sheets and formatting"""
    try:
        logger.info(f"Creating Excel report: {marking_scheme_excel_file}")
        
        # Create backup of existing file if it exists
        if os.path.exists(marking_scheme_excel_file):
            backup_file = f"{marking_scheme_excel_file}.backup.{datetime.now().strftime('%Y%m%d_%H%M%S')}"
            shutil.copy2(marking_scheme_excel_file, backup_file)
            logger.info(f"✓ Created backup: {backup_file}")
        
        # Ensure output directory exists
        os.makedirs(os.path.dirname(marking_scheme_excel_file), exist_ok=True)
        
        # Create Excel writer with multiple sheets
        with pd.ExcelWriter(marking_scheme_excel_file, engine='openpyxl') as writer:
            # Sheet 1: Marking Scheme (detailed rubric)
            df.to_excel(writer, sheet_name='Marking Scheme', index=False)
            logger.info("✓ Created 'Marking Scheme' sheet")
            
            # Sheet 2: Summary with statistics
            summary_data = {
                'Metric': [
                    'Total Questions',
                    'Total Marks',
                    'Average Marks per Question',
                    'Min Marks per Question',
                    'Max Marks per Question',
                    'Verification Feedback',
                    'Generated On',
                    'Input File',
                    'Output File'
                ],
                'Value': [
                    total_questions,
                    total_marks,
                    f"{df['marks'].mean():.1f}",
                    df['marks'].min(),
                    df['marks'].max(),
                    verification_feedback,
                    datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    os.path.basename(marking_scheme_word_file),
                    os.path.basename(marking_scheme_excel_file)
                ]
            }
            summary_df = pd.DataFrame(summary_data)
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            logger.info("✓ Created 'Summary' sheet")
            
            # Sheet 3: Question Overview (simplified view)
            overview_df = df[['question_number', 'question_text', 'marks']].copy()
            overview_df.to_excel(writer, sheet_name='Question Overview', index=False)
            logger.info("✓ Created 'Question Overview' sheet")
            
            # Sheet 4: Validation Report
            validation_data = {
                'Check': [
                    'All questions have marking schemes',
                    'All questions have valid marks',
                    'Question numbers are unique',
                    'Total marks calculated',
                    'General grading guide processed'
                ],
                'Status': [
                    '✅ PASS' if all(q.get('marking_scheme', '').strip() for q in questions_data) else '❌ FAIL',
                    '✅ PASS' if all(isinstance(q.get('marks', 0), int) and q.get('marks', 0) > 0 for q in questions_data) else '❌ FAIL',
                    '✅ PASS' if len(set(q.get('question_number', '') for q in questions_data)) == len(questions_data) else '⚠️ WARNING',
                    f'✅ PASS ({total_marks} marks)',
                    '✅ PROCESSED' if general_guide else 'ℹ️ NOT FOUND'
                ]
            }
            validation_df = pd.DataFrame(validation_data)
            validation_df.to_excel(writer, sheet_name='Validation', index=False)
            logger.info("✓ Created 'Validation' sheet")
            
            # Sheet 5: Content Verification (Grounding)
            if verification_df is not None and not verification_df.empty:
                verification_df.to_excel(writer, sheet_name='Content Verification', index=False)
                logger.info("✓ Created 'Content Verification' sheet")
        
        # Verify file was created successfully
        if os.path.exists(marking_scheme_excel_file):
            file_size = os.path.getsize(marking_scheme_excel_file)
            logger.info(f"✓ Excel file created successfully ({file_size:,} bytes)")
            return True
        else:
            raise FileNotFoundError("Excel file was not created")
            
    except Exception as e:
        logger.error(f"❌ Excel export failed: {e}")
        raise

def create_verification_word_report(verification_df, verification_feedback, output_path):
    """Create a Word report for the verification results"""
    try:
        logger.info(f"Creating Word verification report: {output_path}")
        doc = Document()
        doc.add_heading('Marking Scheme Verification Report', 0)
        
        # General Feedback
        doc.add_heading('General Feedback', level=1)
        if verification_feedback:
            p = doc.add_paragraph()
            add_formatted_text(p, verification_feedback)
        else:
            doc.add_paragraph("No general feedback available.")
            
        # Question Feedback
        doc.add_heading('Question Verification', level=1)
        
        if verification_df is not None and not verification_df.empty:
            for _, row in verification_df.iterrows():
                q_num = row.get('question_number', 'Unknown Question')
                is_correct = row.get('is_correct', False)
                feedback = row.get('feedback', '')
                suggestion = row.get('suggestion', '')
                
                heading = doc.add_heading(f"{q_num}", level=2)
                
                # Status
                p = doc.add_paragraph()
                run = p.add_run(f"Status: {'✅ Correct' if is_correct else '❌ Incorrect'}")
                run.bold = True
                if not is_correct:
                    run.font.color.rgb = RGBColor(255, 0, 0)

                # Feedback
                doc.add_heading('Feedback:', level=3)
                p = doc.add_paragraph()
                add_formatted_text(p, feedback)
                
                # Suggestion
                if suggestion and suggestion != 'None':
                    doc.add_heading('Suggestion:', level=3)
                    p = doc.add_paragraph()
                    add_formatted_text(p, suggestion)
                
                doc.add_paragraph("_" * 40) # Separator
        else:
            doc.add_paragraph("No question-level verification data available.")
            
        doc.save(output_path)
        logger.info(f"✓ Word report created successfully")
        return True
        
    except Exception as e:
        logger.error(f"❌ Word report creation failed: {e}")
        return False

# Execute Excel export
try:
    # Check if v_df exists (from previous cell)
    if 'v_df' not in locals():
        v_df = None
    if 'verification_general_feedback' not in locals():
        verification_general_feedback = ""
        
    success = create_excel_report(df, marking_scheme_excel_file, total_questions, total_marks, v_df, verification_general_feedback)
    
    if success:
        # Also create Word report for verification
        if v_df is not None or verification_general_feedback:
            word_report_file = marking_scheme_excel_file.replace('.xlsx', '_verification.docx')
            create_verification_word_report(v_df, verification_general_feedback, word_report_file)
            print(f"📁 Verification Word report: {word_report_file}")

        print("\n" + "="*60)
        print(colored("🎉 STEP 2 COMPLETED SUCCESSFULLY!", "green", attrs=['bold']))
        print("="*60)
        print(f"📁 Output file: {marking_scheme_excel_file}")
        print(f"📊 Questions processed: {total_questions}")
        print(f"📊 Total marks: {total_marks}")
        print(f"📋 Sheets created: Marking Scheme, Summary, Question Overview, Validation, Content Verification")
        print(f"⏰ Processing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
        print("\n✅ Ready for Step 3: Question Annotations")
except Exception as e:
    print(f"❌ Excel export failed: {e}")
    raise

2026-01-07 14:20:19,481 - INFO - Creating Excel report: ../sample/VTC Test Marking Scheme.xlsx
2026-01-07 14:20:19,489 - INFO - ✓ Created backup: ../sample/VTC Test Marking Scheme.xlsx.backup.20260107_142019
2026-01-07 14:20:19,798 - INFO - ✓ Created 'Marking Scheme' sheet
2026-01-07 14:20:19,804 - INFO - ✓ Created 'Summary' sheet
2026-01-07 14:20:19,808 - INFO - ✓ Created 'Question Overview' sheet
2026-01-07 14:20:19,813 - INFO - ✓ Created 'Validation' sheet
2026-01-07 14:20:19,816 - INFO - ✓ Created 'Content Verification' sheet
2026-01-07 14:20:19,862 - INFO - ✓ Excel file created successfully (12,401 bytes)
2026-01-07 14:20:19,866 - INFO - Creating Word verification report: ../sample/VTC Test Marking Scheme_verification.docx
2026-01-07 14:20:19,981 - INFO - ✓ Word report created successfully


📁 Verification Word report: ../sample/VTC Test Marking Scheme_verification.docx

🎉 STEP 2 COMPLETED SUCCESSFULLY!
📁 Output file: ../sample/VTC Test Marking Scheme.xlsx
📊 Questions processed: 5
📊 Total marks: 50
📋 Sheets created: Marking Scheme, Summary, Question Overview, Validation, Content Verification
⏰ Processing completed at: 2026-01-07 14:20:19

✅ Ready for Step 3: Question Annotations
